In [5]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from keras.models import load_model

file_path = '/content/Housing (3).csv'
data = pd.read_csv(file_path)

X = data.drop('price', axis=1)
y = data['price']

categorical_columns = ['mainroad', 'guestroom', 'basement', 'hotwaterheating',
                       'airconditioning', 'prefarea', 'furnishingstatus']
numerical_columns = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

model = Sequential()
model.add(Dense(15, activation='tanh', input_dim=X_train.shape[1]))
model.add(Dense(20, activation='tanh'))
model.add(Dense(15, activation='tanh'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer=SGD(), metrics=['mean_squared_error'])

history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

model.save('housing_price_model.h5')

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print(f"Training Mean Squared Error: {train_mse}")
print(f"Testing Mean Squared Error: {test_mse}")

loaded_model = load_model('housing_price_model.h5')
def predict_housing_price(features):
    # Create a DataFrame with user input for correct preprocessing
    user_df = pd.DataFrame([features], columns=['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus'])

    features_scaled = preprocessor.transform(user_df) # Use transform on the DataFrame
    prediction = loaded_model.predict(features_scaled)
    return prediction[0][0]

user_input = [8000, 3, 2, 2, 1, 'yes', 'no', 'no', 'no', 'yes', 'yes', 'furnished'] # Correct user input
predicted_price = predict_housing_price(user_input)
print(f"Predicted Housing Price: {predicted_price}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/Housing (3).csv'